#### Import important libraries

In [12]:
# Import important libraries

import pandas as pd

#Read reports
orderReport = pd.read_csv('Order Report.csv')
skuMaster = pd.read_csv('SKU Master.csv')
pinCode = pd.read_csv('pincodes.csv')
invoice = pd.read_csv('Invoice.csv')
courierRates = pd.read_csv('Courier Company - Rates.csv')

#### View  Dataset Reports

In [17]:
# View part of the reports

print(f"\n\nOrder Report\n\n {orderReport.head()} \n")
print(f"\nSKU Master\n\n {skuMaster.head()} \n")
print(f"\nPin Codes Mapping\n\n {pinCode.head()} \n")
print(f"\nInvoices Report\n\n {invoice.head()} \n")
print(f"\nCourier Rate Report\n\n {courierRates.head()} \n")



Order Report

    ExternOrderNo            SKU  Order Qty  Unnamed: 3  Unnamed: 4
0     2001827036  8904223818706        1.0         NaN         NaN
1     2001827036  8904223819093        1.0         NaN         NaN
2     2001827036  8904223819109        1.0         NaN         NaN
3     2001827036  8904223818430        1.0         NaN         NaN
4     2001827036  8904223819277        1.0         NaN         NaN 


SKU Master

              SKU  Weight (g)  Unnamed: 2  Unnamed: 3  Unnamed: 4
0  8904223815682         210         NaN         NaN         NaN
1  8904223815859         165         NaN         NaN         NaN
2  8904223815866         113         NaN         NaN         NaN
3  8904223815873          65         NaN         NaN         NaN
4  8904223816214         120         NaN         NaN         NaN 


Pin Codes Mapping

    Warehouse Pincode  Customer Pincode Zone  Unnamed: 3  Unnamed: 4
0             121003            507101    d         NaN         NaN
1             12

#### Look for Balnks or Missing Values

In [16]:
# Look at the missing values
print(f"\nMissing Values in the Website Order Report:\n\n{orderReport.isnull().sum()} \n")
# Look at the missing values
print(f"\nMissing Values in the Website SKU Master Report:\n\n{skuMaster.isnull().sum()} \n")
# Look at the missing values
print(f"\nMissing Values in the Website Invoices Report:\n\n{invoice.isnull().sum()} \n")
# Look at the missing values
print(f"\nMissing Values in the Website Pincode Mapping Report:\n\n{pinCode.isnull().sum()} \n")
# Look at the missing values
print(f"\nMissing Values in the Website Courier Rates Report:\n\n{courierRates.isnull().sum()} \n")


Missing Values in the Website Order Report:

ExternOrderNo      0
SKU                0
Order Qty          0
Unnamed: 3       400
Unnamed: 4       400
dtype: int64 


Missing Values in the Website SKU Master Report:

SKU            0
Weight (g)     0
Unnamed: 2    66
Unnamed: 3    66
Unnamed: 4    66
dtype: int64 


Missing Values in the Website Invoices Report:

AWB Code                0
Order ID                0
Charged Weight          0
Warehouse Pincode       0
Customer Pincode        0
Zone                    0
Type of Shipment        0
Billing Amount (Rs.)    0
dtype: int64 


Missing Values in the Website Pincode Mapping Report:

Warehouse Pincode      0
Customer Pincode       0
Zone                   0
Unnamed: 3           124
Unnamed: 4           124
dtype: int64 


Missing Values in the Website Courier Rates Report:

fwd_a_fixed         0
fwd_a_additional    0
fwd_b_fixed         0
fwd_b_additional    0
fwd_c_fixed         0
fwd_c_additional    0
fwd_d_fixed         0
fwd_d_a